In [8]:
from openai import OpenAI


client = OpenAI(base_url="http://localhost:11434/v1", api_key="dummy")
# Example usage
response = client.chat.completions.create(
    model="gemma3",
    messages=[
        {"role": "user", "content": "Hello! How are you?"},
        {"role": "assistant", "content": "I'm fine, thank you! How can I assist you today?"}
    ],
)

In [ ]:
from typing import Any, AsyncGenerator, Generator
from openai import AsyncOpenAI

async def acreate() -> AsyncGenerator[str, None]:
    async_client = AsyncOpenAI(base_url="http://localhost:11434/v1", api_key="dummy")
    response = await async_client.chat.completions.create(
        model="gemma3",
        messages=[
            {"role": "user", "content": "Hello! How are you?"},
            {"role": "assistant", "content": "I'm fine, thank you! How can I assist you today?"}
        ],
        stream=True,
    )
    async for chunk in response:
        yield {"state": "generationg", "content": chunk.choices[0].delta.content}

async def main() -> AsyncGenerator[str, None]:
    yield {"state": "accepted", "content": "hello world"}
    response = await acreate()
    for chunk in response:
        yield chunk
    yield {"state": "completed", "content": "goodbye world"}

In [39]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

app = FastAPI()

@app.get("/stream")
async def stream():
    async def event_generator():
        async for item in main():
            yield f"data: {item}\n\n"
    return StreamingResponse(event_generator(), media_type="text/event-stream")